<h1>ECE 445: Machine Learning for Engineers - Albert Tran</h1>
<h2>Mini Jupyter Exercise #4</h2>
<h3>k-Nearest Neighbor Classification</h3>
<br>
<strong>Objective:</strong> In this exercise, we will engage in classification of images of handwritten digits ‘0’ and ‘1’ using
k-nearest neighbor (k-NN) classification.
<br>
<h4>Training Data</h4>

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import math

from sklearn.datasets import load_digits
images, labels = load_digits(2, return_X_y = True)

# Labeled training set
training_images = images[:300]
training_labels = labels[:300]

# Labeled test set
test_images = images[300:]
test_labels = labels[300:]

<h4>k-NN Classification Using 2-D Features</h4>